# Audio Classification Using Edge Impulse
This example uses audio already recorded from fast analog input IN2 and uses local Edge Impulse Classification to classify 5 seconds of audio.

This is based on the Edge Immpulse example:
https://github.com/edgeimpulse/linux-sdk-python/blob/master/examples/audio/classify.py 

The following are required to use the edge_impulse_linux (they can be run from a Jupyter code cell):  
`!apt update`

`!apt install libatlas-base-dev libportaudio2 libportaudiocpp0 portaudio19-dev python3-opencv`

`!apt install libatlas-base-dev libportaudio2 libportaudiocpp0 portaudio19-dev python3-opencv`

**Reload the Jupyter kernel** then run:

`!pip3 install PyAudio`

`!pip3 install edge_impulse_linux`

## Import Libraries

In [ ]:
from redpitaya.overlay.mercury import mercury as overlay
import IPython.display as ipd
import matplotlib.pyplot as plt

#Audio processing requirements
import os
import sys, getopt
import signal
import numpy as np
from scipy.io import wavfile

#Edge Impulse requirements
import json
import time, hmac, hashlib
import requests
import re, socket
import math
from edge_impulse_linux.runner import ImpulseRunner

fpga = overlay()

## Define Functions

In [ ]:
def record_audio():
    # synchronization and trigger sources are the default,
    # which is the module itself
    in2.reset()
    in2.start()
    in2.trigger()
    print ('Recording started')
    # wait for data
    while (in2.status_run()): pass
    print ('Recording complete')
    data = in2.data(N)
    return data[1:] #Remove first sample as it maybe zero

In [ ]:
def display_audio(audio_data,sr = 3000): #default sample rate 3 Ksps
    return ipd.Audio(audio_data, rate=sr) # load a NumPy array

In [ ]:
def plot_audio(audio_data):
    # show the part of the buffer requested by pre/post trigger timing - in this example the whole buffer.
    plt.plot(audio_data_returned)
    plt.show()

In [ ]:
def save_audio(audio_data, file='float_wave_test_raw.wav', sr = 3000): #default sample rate 3 Ksps
    wavfile.write(file, sr, audio_data)

## Setup Variables

In [ ]:
in2 = fpga.osc(1, 1.0)
# data rate decimation 
in2.decimation = 41667 #125 Msps / 41667 = 30 Ksps

# trigger timing [sample periods]
N = in2.buffer_size
in2.trigger_pre  = 0
in2.trigger_post = N

# disable hardware trigger sources - the Jupyter notebook will trigger the start of audio recording
in2.trig_src = 0

In [ ]:
runner = None
model_file_name = 'modelfile.eim'
audio_file_name = 'float_wave_test_raw.wav'

dir_path = os.path.dirname(os.path.realpath(model_file_name))
modelfile = os.path.join(dir_path, model_file_name)
audio_file = os.path.join(dir_path, audio_file_name)

In [ ]:
samplerate, generator = wavfile.read(audio_file)

features = np.array([], dtype=np.int16)
runner = ImpulseRunner(modelfile)

model_info = runner.init()
labels = model_info['model_parameters']['labels']
window_size = model_info['model_parameters']['input_features_count']
sampling_rate = model_info['model_parameters']['frequency']
print('Loaded runner for "' + model_info['project']['owner'] + ' / ' + model_info['project']['name'] + '"')
print('labels: ', labels)
print('window_size: ', window_size)
print('sampling_rate: ', sampling_rate)

## Recording the Audio and Classifying it

In [ ]:
audio_data_returned = record_audio()
save_audio(audio_data_returned, sr = 3000)

samplerate, generator = wavfile.read(audio_file)
features = np.array([], dtype=np.int16) #Reset features
features = np.concatenate((features, generator), axis=0)

#Reset the runner
runner = None
runner = ImpulseRunner(modelfile)
model_info = runner.init()
res = runner.classify(features[:window_size].tolist())

#Show the classification key that has the highest confidence index
best_label = max(res['result']['classification'], key=lambda key: res['result']['classification'][key])
confidence = res['result']['classification'][best_label]
print(best_label,"{:.2%}".format(confidence))

## Classification Detailed Results

In [ ]:
res

In [ ]:
print('Result (%d ms.) ' % (res['timing']['dsp'] + res['timing']['classification']), end='')
for label in labels:
    score = res['result']['classification'][label]
    print('%s: %.2f\t' % (label, score), end='')
    print('', flush=True)

## Display the Audio Waveform

In [ ]:
plot_audio(audio_data_returned)
display_audio(audio_data_returned)